In [21]:
import sys
import pathlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.append(str(pathlib.Path.cwd()))

from Classes.Func.KitTools import SaveGen

folder_path = r"C:\Main\Data\_\Result\Mix\20221114_20_pcv_var\Table"
save_path = SaveGen(pathlib.Path(folder_path).parent, "result")
save_path.mkdir(parents=True, exist_ok=True)
file_list = [i for i in pathlib.Path(folder_path).iterdir() if i.is_file() and i.suffix == ".csv"]

In [22]:
class HRV():
    def __init__(self, resp_ind:np.ndarray, resp_wid:np.ndarray, scale_st:int = 300) -> None:
        self.__s_st = scale_st
        self.__ind = self.__TypeJudge(resp_ind)
        self.__wid = self.__TypeJudge(resp_wid)
        self.__sdnn = self.__SDNN()
        self.__sdann = self.__SDANN()
        self.__sdnni = self.__SDNNI()

    @property
    def sdnn(self):
        return self.__sdnn

    @property
    def sdann(self):
        return self.__sdann

    @property
    def sdnni(self):
        return self.__sdnni

    def __TypeJudge(self, arr):
        arr if type(arr) == np.ndarray else np.array(arr)
        return arr

    def __SDNN(self):
        sdnn = np.std(self.__ind)
        return np.round(sdnn, 4)

    def __SDANN(self):
        wid_scale_i = [0]
        for i in range(len(self.__wid)):
            if sum(self.__wid[wid_scale_i[-1]:i]) < self.__s_st:
                continue
            else:
                wid_scale_i.append(i)
        sdann = [np.std(self.__ind[wid_scale_i[i]:wid_scale_i[i+1]]) for i in range(len(wid_scale_i)-1)]
        sdann = np.round(np.array(sdann), 4)
        return sdann

    def __SDNNI(self):
        sdann = self.__SDANN()
        sdnni = np.round(np.mean(sdann),4)
        return sdnni

class SavePlots():
    def __init__(self, save_path:pathlib.Path) -> None:
        self.__s_fold = save_path
        self.__s_fold.mkdir(parents=True, exist_ok=True)

    def SDANNPlot(self, sdann:np.ndarray, fig_title:str, save_name:str, **kwargs):
        sns.set_theme(style="whitegrid")
        fig, ax = plt.subplots(1, 1, **kwargs)
        ax.plot(sdann, "b-", linewidth=2)
        ax.plot(sdann, "ro")
        ax.set_title(fig_title)
        fig.tight_layout()
        fig.savefig(self.__s_fold/(save_name+".png"), dpi=300)
        plt.close()

    def LinePlot(self, data:pd.DataFrame, v_:str,i_:str, save_name:str, **kwargs):
        sns.set_theme(style="whitegrid")
        fig, ax = plt.subplots(1, 1, **kwargs)
        sns.lineplot(data=data, x=i_, y=v_, ax=ax)
        fig.tight_layout()
        fig.savefig(self.__s_fold/(save_name+".png"), dpi=300)
        plt.close()


In [23]:
scale_range = [60, 180, 300, 600, 1200, 1800, 3600]
inds_select = ["rr", "v_t_i", "ve", "rsbi", "mp_jb_d", "mp_jm_d", "mp_jl_d"]

def OutliersWipe(df:pd.DataFrame, inds:list, max_d_st: int = 3) -> None:
    df_ = df.copy()
    outlier = lambda arr, max_d: (arr - np.mean(arr)) > max_d * np.std(arr)
    for ind in inds:
            df_[ind + '_val'] = ~outlier(df[ind], max_d_st)
    df_val = df_[df_[[ind + '_val' for ind in inds]].all(axis=1)]
    df_val = df.iloc[df_val.index]
    
    return df_val

for file in file_list:
    pid = file.name.split('_')[0]
    save_p = save_path / str(pid)
    fig_p = SavePlots(save_p)
    df = OutliersWipe(pd.read_csv(file), inds_select)
    for s in scale_range:
        for ind in inds_select:
            hrv_p = HRV(df[ind].tolist(),df.wid.tolist(),s)
            save_n = "{0}_{1}_{2}".format(ind, str(s).rjust(len(str(scale_range[-1])), '0'), hrv_p.sdnn)
            title_n = "{0}, S={1}, SDNN={2}, SDNNI={3}".format(ind, s, hrv_p.sdnn, hrv_p.sdnni)
            fig_p.SDANNPlot(hrv_p.sdann, title_n, save_n, figsize=(10,4))
            fig_p.LinePlot(df, ind, "t_ind", ind+"_raw", figsize=(15,5))




In [ ]:
pd.read_csv(file_list[0]).t_ind